In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import modAL
import numpy as np
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

In [ ]:
df=pd.read_csv("C:\\users/rsaur/Downloads/project/NF-UNSW-NB15 -Dataset/NF-UNSW-NB15 -Dataset.csv")
df

In [ ]:
df['Attack'].value_counts()

In [ ]:
df1=df.copy()
df1

In [ ]:
x=df1.drop(["Label","Attack"],axis=1)
x

In [ ]:
y=df1[["Label"]]
y

In [ ]:
y.value_counts()

In [ ]:
y.shape,x.shape

In [ ]:
label_encoder=LabelEncoder()
x["IPV4_SRC_ADDR"]=label_encoder.fit_transform(x["IPV4_SRC_ADDR"])
x["IPV4_DST_ADDR"]=label_encoder.fit_transform(x["IPV4_DST_ADDR"])

In [ ]:
x["IPV4_SRC_ADDR"].value_counts(), x["IPV4_DST_ADDR"].value_counts()

In [ ]:
smote = SMOTE(sampling_strategy= 'minority')
x_new,y_new = smote.fit_resample(x,y)
y_new.value_counts()

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
x_scaled=scaler.fit_transform(x_new)
x_scaled

In [ ]:
x_train,x_pool,y_train,y_pool=train_test_split(x_scaled,y_new,train_size=0.1,random_state=42,shuffle=True,stratify=y_new)

In [ ]:
def model_ANN():
    model= keras.Sequential([
        keras.layers.Dense(50,activation='relu',input_shape=(12,)),
        keras.layers.Dense(50,activation='relu',input_shape=(12,)),
        keras.layers.Dense(1,activation='sigmoid')
])
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    #model.fit(x_train,y_train,batch_size=100,epochs=10)
    return model
classifier=KerasClassifier(model_ANN())

In [ ]:
learner= ActiveLearner(
    estimator=classifier,
    query_strategy=uncertainty_sampling,
    x_training=x_train,
    y_training=y_train,
    verbose=1
)

In [ ]:
n_queries=30
for idx in range(n_queries):
    query_idx ,query_sample = learner.query(x_pool,n_instances=100,verbose=0)
    learner.teach(x_pool,y_pool,only_new=True,verbose=1)
    x_pool=np.delete(x_pool,query_idx,axis=0)
    y_pool=np.delete(y_pool,query_idx,axis=0)

In [ ]:
score=learner.score(x_pool,y_pool )
print("score:",score*100,"%")

In [ ]:
y_pred=learner.predict(x_pool)

In [ ]:
category_names=["Benign","Attack"]
report=classification_report(y_pool,y_pred,target_names=category_names,labels=[0,1],digits=4)
print(report)

In [ ]:
confusion_matrix=ConfusionMatrixDisplay.from_predictions(y_pool,y_pred)
confusion_matrix
print("Benign = 0")
print("Attack = 1")